In [1]:
import pandas as pd
import numpy as np

import time
from tqdm import tqdm

In [2]:
df_kt = pd.read_csv('./KT_data/dp_frc_bldg_ftc_pred_slng_m_sum_202205.csv',sep='|')

In [4]:
len(df_kt)/43

1366887.0

### kt data

In [44]:
# 필요한 컬럼 선택
df_kt = df_kt[['bd_mgt_sn','ftc_cate2_cd','pred_slng_amt']]

# 전체 컬럼명 변경
df_kt.columns = ['건물관리번호','업종코드','예상매출액']

# bd_mgt_sn : 건물관리번호
# ftc_cate2_cd : 업종코드
# pred_slng_amt : 예상매출액

In [43]:
df_kt = df_kt[:1000]

In [45]:
df_kt_ = df_kt

In [35]:
import time

In [36]:
# 금액 > 한글-숫자로 타입 변경
# 방법1. 레이블로 찾기 -> 더 빠름
start = time.time()

df_kt.loc[df_kt['예상매출액'].str.contains('억'), '예상매출액'] = '100000000'
df_kt.loc[df_kt['예상매출액'].str.contains('만'), '예상매출액'] = '1000000'
df_kt['예상매출액'] = df_kt['예상매출액'].astype(float)

elapsed_time = time.time() - start
print(f'elapsed_time : {elapsed_time}')

elapsed_time : 0.009577274322509766


In [46]:
# 방법2. 인덱스로 찾기
start = time.time()

idx = df_kt[df_kt['예상매출액'].str.contains('억')].index
df_kt.iloc[idx,2] = '100000000'

elapsed_time = time.time() - start
print(f'elapsed_time : {elapsed_time}')

elapsed_time : 0.010125875473022461


In [48]:
# 제외 업종 삭제
list_ = ['A03', 'A10', 'B01', 'B03', 'B12', 'B13', 'B17', 'B20', 'B21', 'C01']

In [49]:
df_kt = df_kt[~df_kt['업종코드'].isin(list_)]

elapsed_time : 0.006770133972167969


In [91]:
# 삭제 확인
len(df_kt['업종코드'])/33

1366887.0

In [92]:
df_kt.to_csv('./df_kt_예상매출액_수정본.csv')

### 값 추출

In [93]:
# 0. sort_values > groupby > head
df_kt_ = df_kt.sort_values(by=['건물관리번호','예상매출액'], ascending=False)
df_kt_

,건물관리번호,업종코드,예상매출액
47362773,5013032026119420001000001,A04,34862315.19
47362780,5013032026119420001000001,B19,33172325.17
47362768,5013032026119420001000001,C04,26536268.50
47362766,5013032026119420001000001,A11,19667134.53
47338316,5013032026119420001000001,A02,19616291.27
...,...,...,...
32616844,1111010100100010000030844,C03,5680882.66
32616849,1111010100100010000030844,B14,3712487.19
32618355,1111010100100010000030844,B02,1471112.23
32616830,1111010100100010000030844,B08,1000000.00


In [94]:
df_kt_ = df_kt_.groupby(['건물관리번호']).head(3)

In [95]:
df_kt_

,건물관리번호,업종코드,예상매출액
47362773,5013032026119420001000001,A04,34862315.19
47362780,5013032026119420001000001,B19,33172325.17
47362768,5013032026119420001000001,C04,26536268.50
12108610,5013032026117900000000003,B08,45081262.26
12107081,5013032026117900000000003,A04,44926938.52
...,...,...,...
27063647,1111010100100030065030970,C06,63355986.00
27063631,1111010100100030065030970,A01,36687660.44
32616841,1111010100100010000030844,C06,70822503.46
32616831,1111010100100010000030844,B19,56420288.45


In [100]:
df_kt_.to_csv('./df_kt_Top3.csv', index=False, encoding='utf-8-sig')

In [101]:
df = pd.read_csv('./df_kt_Top3.csv')

In [102]:
df

,건물관리번호,업종코드,예상매출액
0,5013032026119420001000001,A04,34862315.19
1,5013032026119420001000001,B19,33172325.17
2,5013032026119420001000001,C04,26536268.50
3,5013032026117900000000003,B08,45081262.26
4,5013032026117900000000003,A04,44926938.52
...,...,...,...
4100656,1111010100100030065030970,C06,63355986.00
4100657,1111010100100030065030970,A01,36687660.44
4100658,1111010100100010000030844,C06,70822503.46
4100659,1111010100100010000030844,B19,56420288.45


In [35]:
# 1. groupby 조건 두 개 주기 -> 그룹바이하면 인덱스 순서로 묶이기 때문에, 그 전에 정렬한 대로 묶이지 않음
df_kt_group = df_kt.groupby(['건물관리번호', '업종코드']).sum('예상매출액')

In [103]:
df_kt_group.reset_index()

,건물관리번호,업종코드,예상매출액
0,1111010100100010000030844,A01,28838829.92
1,1111010100100010000030844,A02,32194237.70
2,1111010100100010000030844,A04,11940900.28
3,1111010100100010000030844,A05,43223687.05
4,1111010100100010000030844,A06,26287068.16
...,...,...,...
45107266,5013032026119420001000001,C03,3961296.20
45107267,5013032026119420001000001,C04,26536268.50
45107268,5013032026119420001000001,C05,6052737.21
45107269,5013032026119420001000001,C06,11137264.96


In [49]:
# 2. 매출 상위 정렬
df_kt_sort = df_kt_group.sort_values(by='예상매출액', ascending=False)

In [50]:
df_kt_sort

,,예상매출액
건물관리번호,업종코드,
1174010100100470012018747,B09,100000000.0
1138010400103940018034166,C07,100000000.0
4513013500105030011026758,A05,100000000.0
1165010800116570006023466,A13,100000000.0
2671031030101000002013067,C02,100000000.0
...,...,...
4790025021100740005043191,B07,1000000.0
2717010300115420011020711,B02,1000000.0
1153010200100290011019374,B06,1000000.0


In [51]:
df_kt_head = df_kt_sort.groupby(['건물관리번호','업종코드']).head(3)

In [52]:
df_kt_head

,,예상매출액
건물관리번호,업종코드,
1174010100100470012018747,B09,100000000.0
1138010400103940018034166,C07,100000000.0
4513013500105030011026758,A05,100000000.0
1165010800116570006023466,A13,100000000.0
2671031030101000002013067,C02,100000000.0
...,...,...
4790025021100740005043191,B07,1000000.0
2717010300115420011020711,B02,1000000.0
1153010200100290011019374,B06,1000000.0


In [53]:
df_kt_head.groupby(['건물관리번호','업종코드']).sum('예상매출액')

예상매출액
건물관리번호                    업종코드             
1111010100100010000030844 A01   28838829.92
                          A02   32194237.70
                          A04   11940900.28
                          A05   43223687.05
                          A06   26287068.16
...                                     ...
5013032026119420001000001 C03    3961296.20
                          C04   26536268.50
                          C05    6052737.21
                          C06   11137264.96
                          C07   17710573.65

[45107271 rows x 1 columns]

In [37]:
# 한 건물의 매출 상위 업종 3순위
df_kt_sort.loc['4159012700102710000000003'].astype(float).sort_values('예상매출액', ascending=False)[:3]

,예상매출액
업종코드,
C06,89613368.94
B08,89302638.86
B11,77869598.19


In [ ]:
# 뽑은 값 담기 -> 너무 오래걸림 -> 값 정리 후 다시 시도 -> 시간 확인
index_ = []
list_  = []
code   = []

for i in tqdm(range(len(df_kt_sort.index))):
    index_.append(df_kt_sort.index[i][0])
    list_.append(df_kt_sort.loc[df_kt_sort.index[i][0]].astype(float).sort_values('예상매출액', ascending=False)[:3])
    code.append(df_kt_sort.loc[df_kt_sort.index[0][0]].astype(float).sort_values('예상매출액', ascending=False)[:3].index)

In [ ]:
# 결과만 한 데이터 프레임에 담기
a = pd.DataFrame({
    '건물번호':index_
    ,'3순위':list_
    ,'업종':code
})

In [120]:
df_groupby = df_kt.groupby(by=['건물관리번호','업종코드']).sum('예상매출액')

In [119]:
df_groupby['예상매출액'].value_counts()

100만원 미만       9894526
1억원 초과          884833
5384990.10       69271
9369407.39       61588
3728854.73        6714
                ...   
5266400.70           1
15659430.94          1
33652422.69          1
12960113.77          1
17710573.65          1
Name: 예상매출액, Length: 42379321, dtype: int64

### 시각화

In [115]:
import matplotlib.pyplot as plt

In [116]:
# 시각화는 숫자 데이터만 가능
df_groupby.plot(kind='bar')
plt.show()

TypeError: no numeric data to plot

### 멀티인덱스 값 조회

In [83]:
# 컬럼별 값 조회
DF.loc[DF.index,'col3']

col1  col2
a     A       1
      B       2
b     C       3
      D       4
Name: col3, dtype: object

In [85]:
# 특정 값 조회
DF.loc[DF.index[0],'col3']

'1'

In [91]:
DF.loc[DF.index[0],:]['col3']

'1'

In [144]:
DF

col3
col1 col2     
a    A       1
     B       2
b    C       3
     D       4

In [ ]:
DF.loc[('a'),:].head(2)

In [142]:
df = pd.DataFrame()

In [143]:
# 결과 -> 새 데이터 프레임에 추가하기
df.append(DF.loc[('a'),:].head(2))

C:\Users\USER\AppData\Local\Temp\ipykernel_13616\2383382394.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(DF.loc[('a'),:].head(2))


,col3
col2,
A,1
B,2


### pivot_table

In [98]:
DF

col3
col1 col2     
a    A       1
     B       2
b    C       3
     D       4

In [96]:
pd.pivot_table(DF,values='col3',index='col1',columns='col2')

col2,A,B,C,D
col1,,,,
a,1.0,2.0,NaN,NaN
b,NaN,NaN,3.0,4.0
